![](Arrow.png)


# Arrow Series: US S&P Index Stock Analysis and Predictions

The Objective of this template is to help you analyze and pick stocks that best suits your Risk and Return preferences. It is always a challenge to select stocks that are high in returns because generally, they are high in risk as well. The python script will make an attempt using Data Science visualization technique scatter plot to compare various stock that offers the same return but with less risk. 

The script will also look and normalized price growth movement, some fundamental analysis and ratios, prediction of stock prices using Brownian Motion Model and at the end will have a look at all stocks in an index to refine the search for the best stocks.

Excel is the world’s most widely used tool by Finance professionals for performing various analysis, considering the fact, all data obtained from yahoo finance and calculations tables will be saved in CSV format, with their respective names for further analysis. Further the images generated in this tempales will also be saved in png format.

The script results can be saved as PDF for reporting purposes, if you wish to not to include code lines in the final report there is an option available the bottom of the script to hide all code lines.

The script is designed with the view that users are not IT professionals and don’t have any coding understanding and experience. 

#### So What you need to run the script:
1.	Understanding of Finance and all Models used in the script.
2.	Ideally Microsoft Azure Notebook account, which is Free at the moment (https://notebooks.azure.com/). Please refer to the following YouTube video for steps on how to create one: https://www.youtube.com/watch?v=r4B_zs3Lb-Y
3.	Log in to Azure Notebook
4.	Upload this script in a newly created project 
5.	Follow the instruction in an explanatory video 

### DISCLAIMER: 
> Quality of analysis depends upon the quality of data, models, and libraries. The template is based on yahoo finance data, python libraries written by third parties, and financial models. Together they do not guarantee 100% desired and/or actual outcomes. User should not consider it as a stand-alone decision-making tool to make monetary and non-monetary decisions.
The author of the template will not accept any liabilities whatsoever for any unexpected outcomes or losses.






### First Step  :Type your stock tickers/symbols
- Please take extra care while selecting start date i.e. all stocks should be on listing on that date, otherwise code will through an ugly error.

In [ ]:
ticker = ['MMM', 'ABT', 'APH','BXP','BLK', 'HRB','CL','EFX','ESS','FDX','AES','AAPL'] # Make sure ticker are within ' '. 

### Date format YYYY,MM.DD. Please make sure that your selected stocks are on listing on your selected Date Range.
### Script will through and Error if not listed.

start_date = '2016-06-29'
end_date = '2019-05-24'

In [ ]:
!pip install pandas_datareader
!pip install yahoofinancials

In [ ]:
import numpy as np
import pandas as pd

from pandas_datareader import data as wb
import matplotlib.pyplot as plt
from datetime import datetime
from yahoofinancials import YahooFinancials as yf
from IPython.display import Markdown, display
from scipy.stats import norm
%matplotlib inline



In [ ]:
def printmd(string, color='navy'):
    colorstr = "<span style='color:{}; font-size: 16px'></spna><span style='font-weight: bold'>{}</span>".format(color, string)
    display(Markdown(colorstr))

In [ ]:
data = pd.DataFrame()

#Extracting price data from Yahoo Finance and only selecting Adjusted Close Price 
data = wb.DataReader(ticker, data_source = 'yahoo', start= start_date, end = end_date)['Adj Close']

# Make sure all scripts have same lenght . not-null and float values
printmd("Data Information")
data.info()
data.to_csv('stock_prices.csv')



### Review data obtained from yahoo finance
#### First 5 historical prices 

In [ ]:
data.head()

#### Last 5 (Current) Prices

In [ ]:
data.tail()

#### Ploting Price Movement

In [ ]:
# Plot and graphs style and size
plt.rcParams['figure.figsize'] = [20, 14]
plt.rcParams['figure.dpi'] = 200
plt.style.use('ggplot')


ax = data.plot(figsize=(18,12),linewidth=1.5, fontsize=8)

# Loop to add column names for each graph line.
for line, name in zip(ax.lines, data.columns):
    y = line.get_ydata()[-1]
    ax.annotate(name, xy=(1,y), xytext=(2,0), color=line.get_color(), 
                xycoords = ax.get_yaxis_transform(), textcoords="offset points",
                size=12, va="center")



# data.plot(figsize=(15,8),linewidth=1, fontsize=6, label = ticker);
plt.xlabel('Year', fontsize=10);


plt.ylabel("Price($) ", fontsize=10)
plt.title("Stocks Price Movement", fontsize=14)
plt.savefig('stock_price_movement.png')
plt.ylim(-10,1000)
plt.show()


# Normalised Price Movement Starting with base 100
Normalised price movment provide a comparison of various securties, considering if all started at the same point, known as base 100, to see who gained/lost in the given period.

In [ ]:
normalization =(data/data.iloc[0]*100)

plt.rcParams['figure.figsize'] = [20,8]
plt.rcParams['font.sans-serif'] = ['Tahoma', 'DejaVu Sans',
                               'Lucida Grande', 'Verdana']
plt.rcParams['font.size'] = 8
plt.rcParams['figure.dpi'] = 200
plt.style.use('ggplot')



ax = normalization.plot(figsize=(18,12),linewidth=1.5, fontsize=8)

for line, name in zip(ax.lines, normalization.columns):
    y = line.get_ydata()[-1]
    ax.annotate(name, xy=(1,y), xytext=(2,0), color=line.get_color(), 
                xycoords = ax.get_yaxis_transform(), textcoords="offset points",
                size=12, va="center")


#plt.ylim(0,1000)
#normalization.plot(figsize=(18,12),linewidth=1, fontsize=6);
plt.xlabel('Year', fontsize=10,color= 'navy');
plt.ylabel("Price Movement Starting from at 100",color= 'navy', fontsize=12)
plt.title("Stocks Price Grwoth", color= 'darkgreen', fontsize=16, fontweight='black')
plt.savefig('stock_price_growth.png')
plt.show()
normalization.to_csv('normalised.csv')

### Review normalised data

#### First 5 rows 

In [ ]:
normalization.head()

#### Last 5 rows

In [ ]:
normalization.tail()

## Risk and Return Analysis:

#### Return:
 -  Return is calculated as percentage price change from prevoius day for all days in dataset.
 -  Mean return for entire daily percentage change i.e average daily return 
 -  Annualised average daily return by * 250 days 

#### Risk:
 - Risk is calculated as daily stadard deviation
 - Annuliased by * 250 days
 - square root of 250 days.
 

In [ ]:
simple_return = data.pct_change()
returns = simple_return.mean() *250
risks = simple_return.std() *250 ** 0.5 # ** 0.5 for taking standard deviation of 250 days
df = pd.DataFrame(risks * 100, columns=['Risk'])
df['Return'] = returns *100
names = df.index
df.insert(0, 'Names', names )


In [ ]:
plt.rcParams['figure.figsize'] = [20,8]
plt.rcParams['font.sans-serif'] = ['Tahoma', 'DejaVu Sans',
                               'Lucida Grande', 'Verdana']
plt.rcParams['font.size'] = 8
plt.rcParams['figure.dpi'] = 200
plt.style.use('ggplot')
fig, ax = plt.subplots()


my_scatter_plot = ax.scatter(
df["Risk"], #x values
df["Return"], # y values
c = 'r',marker='o', s = 20
    

)

ax.set_xlabel("Risk in %", color= 'darkblue', fontsize=10, fontweight = 'bold')
ax.set_ylabel("Return in %",color= 'darkblue', fontsize=10,fontweight = 'bold')
ax.set_title("Stocks Risk Return Relationship",color= 'darkgreen', fontsize=16, fontweight='black')

for _, row in df[["Names","Risk","Return"]].iterrows():
    xy = row[["Risk", "Return"]]
    xytext=  xy 
    ax.annotate(row["Names"], xy, xytext, fontsize=8)

#plt.savefig('enron.png')
my_scatter_plot.figsize=(10,6)
plt.savefig('stock_risk_return.png')
plt.show() #show the plot
printmd(" Risk and return combination sorted for returns, from highest % to lowest %: ")
df[[ "Return","Risk",]].sort_values('Return', ascending=False)

# Fundamental Analysis 

In [ ]:
asx_mix = yf(ticker)
asx_eps            = asx_mix.get_earnings_per_share()
asx_mix_payout     = asx_mix.get_payout_ratio()
asx_mix_pe         =  asx_mix.get_pe_ratio()
asx_mix_div_yield = asx_mix.get_dividend_yield()
asx_avg_annual_yield =   asx_mix.get_annual_avg_div_yield()
asx_mix_div_rate = asx_mix.get_dividend_rate()
asx_mix_5yr_div_yield = asx_mix.get_five_yr_avg_div_yield()
asx_price_sales = asx_mix.get_price_to_sales()
asx_beta = asx_mix.get_beta()


In [ ]:

key_stats = pd.DataFrame(index = ticker)

In [ ]:

key_stats['Risk'] = df["Risk"]
key_stats['Return'] = df["Return"]
key_stats['EPS'] = [x for x in asx_eps.values() ]
key_stats['Div Rate'] = [x for x in asx_mix_div_rate.values() ]
key_stats['Payout Ratio'] = [x for x in asx_mix_payout.values() ] 
key_stats['PE Ratio'] = [x for x in asx_mix_pe.values() ]
key_stats['Div Yield'] = [x for x in asx_mix_div_yield.values() ]
key_stats['Annual avg Div Yield'] = [x for x in asx_mix_div_rate.values() ]
key_stats['5 Yrs Div Yield'] = [x for x in asx_mix_5yr_div_yield.values() ]
key_stats['Price to Sales'] = [x for x in asx_price_sales.values() ]
key_stats['Beta'] = [x for x in asx_beta.values() ]

print ('')

printmd("Following data obtained from Yahoo Finance except for Risk and Return: ")
key_stats



In [ ]:
key_stats.to_csv('key_statistics.csv')

In [ ]:
fig, axarr = plt.subplots(3, 2, figsize=(12, 8))

fig.suptitle("Key Statistics and Ratios as at End Date", fontsize = 14, color='darkgreen', fontfamily='Serif')


# ax.set_xticklabels(key_stats[['PE Ratio','EPS','Div Rate','Payout Ratio','Risk']],rotation=0, fontsize=6)
# ax.set_yticklabels(key_stats[['PE Ratio','EPS','Div Rate','Payout Ratio','Risk']],rotation=0, fontsize=6)

plt.rc('xtick',labelsize=6)
plt.rc('ytick',labelsize=8)



key_stats['Price to Sales'].plot.bar(legend=False,
    ax=axarr[0][0]
)
axarr[0][0].set_title("Price to Sales Ratio", fontsize=8)
axarr[0][0].set_xticklabels(key_stats.index,rotation=0,fontsize=6)
axarr[0][0].legend(fontsize= 'x-small')
axarr[0][0].set_ylabel("Amount", fontsize=8)


key_stats[['EPS','Div Rate']].plot.bar(legend=True,
    ax=axarr[0][1]
)
axarr[0][1].set_title("EPS and Div Rate", fontsize=8)
axarr[0][1].set_xticklabels(key_stats.index,rotation=0,fontsize=6)
axarr[0][1].legend(fontsize= 'x-small')
axarr[0][1].set_ylabel("Amount", fontsize=8)


key_stats['Annual avg Div Yield'].plot.bar(legend=False,
    ax=axarr[1][0]
)
axarr[1][0].set_title("Avg Annual Divdend Yield", fontsize=8)
axarr[1][0].set_xticklabels(key_stats.index,rotation=0,fontsize=6)
axarr[1][0].legend(fontsize= 'x-small')
axarr[1][0].set_ylabel("Amount", fontsize=8)



key_stats['Payout Ratio'].plot.bar(fontsize=9,
    ax=axarr[1][1]
)

axarr[1][1].set_title("Payout Ratio", fontsize=8)
axarr[1][1].set_xticklabels(key_stats.index,rotation=0,fontsize=6)
axarr[1][1].set_ylabel("Ratio", fontsize=8)


key_stats[['Risk','Return']].plot.bar( fontsize=9,legend= True,
    ax=axarr[2][0]
)


axarr[2][0].set_title("Risk and Return", fontsize=8)
axarr[2][0].set_xticklabels(key_stats.index,rotation=0,fontsize=6)
axarr[2][0].set_ylabel("%", fontsize=8)
axarr[2][0].legend(fontsize= 'x-small')




key_stats['PE Ratio'].plot.bar(
    ax=axarr[2][1]
)
axarr[2][1].set_title("PE Ratio", fontsize=8)
axarr[2][1].set_ylabel("Ratio", fontsize=8)
axarr[2][1].set_xticklabels(key_stats.index,rotation=0,fontsize=6)

plt.subplots_adjust(hspace=.6)



# Stock Prediction Using Brownain Motion 

#### Brownian Motion
We are calculating brownain motion using these equations and expressions 
$$
daily\_returns = e^{r}
$$

$$
r = drift + stdev \cdot z
$$


$$
drift = u - \frac{1}{2} \cdot var
$$


In [ ]:
### Type time interval i.e. number of days in the future ---------------------------------------- 

t_intervals = 250 # forecasting for upcoming 300 days 

#------------------------------------------------------------------------------------------------


itterations = 1  # 1 series / scanarios for upcoming 700 days forcasts.


log_returns = np.log(1 + data.pct_change())
u = log_returns.mean()
var = log_returns.var()
drift = u - (0.5 * var)
stdev = log_returns.std()

# .values to convert Series into numpy array.
# norm.ppf() function calculate the z-score / standard deviation of an event / calculate distance between event
# to its mean in standard deviations unit

daily_returns = np.exp(drift.values + stdev.values * norm.ppf(np.random.rand(t_intervals,itterations)))
last_stock_price = data.iloc[-1]
price_list = np.zeros_like(daily_returns)
price_list[0] = last_stock_price

for i in range(1, t_intervals):
    price_list[i] = price_list[i-1] * daily_returns[i]

    price_data = pd.DataFrame(price_list, columns= data.columns)
price_data.columns =  price_data.columns.str.replace('.AX','')


symbol_title = ("Stock Price Predictions")
plt.rcParams['figure.dpi'] = 150

ax = price_data.plot(figsize=(22,12),linewidth=1.5, fontsize=8)

for line, name in zip(ax.lines, price_data.columns):
    y = line.get_ydata()[-1]
    ax.annotate(name, xy=(1,y), xytext=(2,0), color=line.get_color(), 
                xycoords = ax.get_yaxis_transform(), textcoords="offset points",
                size=12, va="center")

plt.xlabel('Days', fontsize=10)
plt.ylabel("Price ($)", fontsize=12)
plt.title(symbol_title, fontsize=18)


plt.show()
printmd('For more detail prediction analysis please refer to OneClick series Monte Carlo and Brownian Motion in Python.')

# Analysing All Stocks in the Index 

In [ ]:
number_from = 1
number_to = 40

In [ ]:
url2 = pd.read_html('https://en.wikipedia.org/wiki/List_of_S%26P_500_companies')

codes = url2[0][0][number_from:number_to]
# codes = table1[1:]
codes

In [ ]:

#Date format YYYY,MM.DD
start_date = '2017-06-29'
end_date = '2019-05-24'


In [ ]:
sp = pd.DataFrame()

#Extracting price data from Yahoo Finance and only selecting Adjusted Close Price 
sp = wb.DataReader(codes, data_source = 'yahoo', start= start_date, end = end_date)['Adj Close']

In [ ]:

sp_all_stocks_statistics= sp.describe()
sp.to_csv('all_index_stocks_price.csv')
sp_all_stocks_statistics.to_csv('all_index_stocks_statistics.csv')
sp_all_stocks_statistics

In [ ]:

### Remove Outlier ----------------------------------------------------------------------------------------------------------------

#asx_200.drop(columns=['EHL'], inplace=True)

###----------------------------------------------------------------------------------------------------------------------------------

simple_return = sp.pct_change()
returns = simple_return.mean() *250
risks = simple_return.std() *250 ** 0.5 # ** 0.5 for taking standard deviation of 250 days
df = pd.DataFrame(risks * 100, columns=['Risk'])
df['Return'] = returns *100
names = df.index
df.insert(0, 'Names', names )

plt.rcParams['figure.figsize'] = [22,10]
plt.rcParams['font.sans-serif'] = ['Tahoma', 'DejaVu Sans',
                               'Lucida Grande', 'Verdana']
plt.rcParams['font.size'] = 8
plt.rcParams['figure.dpi'] = 200
plt.style.use('ggplot')
fig, ax = plt.subplots()


my_scatter_plot = ax.scatter(
df["Risk"], #x values
df["Return"], # y values
c = 'r',marker='o', s = 20
    

)

ax.set_xlabel("Risk in %", color= 'darkblue', fontsize=10, fontweight = 'bold')
ax.set_ylabel("Return in %",color= 'darkblue', fontsize=10,fontweight = 'bold')
ax.set_title("Risk Return Relationship for All Stocks in the Index",color= 'darkgreen', fontsize=16, fontweight='black')

for _, row in df[["Names","Risk","Return"]].iterrows():
    xy = row[["Risk", "Return"]]
    xytext=  xy 
    ax.annotate(row["Names"], xy, xytext, fontsize=8)

#plt.savefig('enron.png')
my_scatter_plot.figsize=(10,6)

plt.show() #show the plot
printmd(" Top 10 Risk and return combination sorted for returns, from highest % to lowest %. ")
df[[ "Return","Risk",]].sort_values('Return', ascending=False).to_csv('all_index_stocks_risk_return.csv')
df[[ "Return","Risk",]].sort_values('Return', ascending=False).head(10)